In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#Dataset from https://grouplens.org/datasets/movielens/

# Importing the dataset
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/6.%20AutoEncoders/Dataset/movies.dat -o movies.dat
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/6.%20AutoEncoders/Dataset/users.dat -o users.dat
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/6.%20AutoEncoders/Dataset/ratings.dat -o ratings.dat
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/6.%20AutoEncoders/Dataset/u1.base -o u1.base
!curl https://raw.githubusercontent.com/muke888/UdemyDeepLearning/master/6.%20AutoEncoders/Dataset/u1.test -o u1.test

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  167k  100  167k    0     0   415k      0 --:--:-- --:--:-- --:--:--  415k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  131k  100  131k    0     0   368k      0 --:--:-- --:--:-- --:--:--  368k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.4M  100 23.4M    0     0  19.3M      0  0:00:01  0:00:01 --:--:-- 19.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1549k  100 1549k    0     0  3645k      0 --:--:-- --:--:-- --:--:-- 3645k
  % Total    % Received % Xferd  Average Speed   Tim

In [0]:
# Importing the Dataset1
# These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
# made by 6,040 MovieLens users who joined MovieLens in 2000.

movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
# MovieID::Title::Genres
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# UserID::Gender::Age::Occupation::Zip-code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
# UserID::MovieID::Rating::Timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [0]:
# Dataset2
# Using 100,000 ratings (1-5) from 943 users on 1682 movies.
# Each user has rated at least 20 movies. 
# Simple demographic info for the users (age, gender, occupation, zip)
# 80%/20% split

# Preparing the training set and the test set
training_setDF = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_setDF, dtype = 'int')
test_setDF = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_setDF, dtype = 'int')

In [7]:
# user id | item id | rating | timestamp.
training_setDF.info()
training_setDF.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79999 entries, 0 to 79998
Data columns (total 4 columns):
1            79999 non-null int64
1.1          79999 non-null int64
5            79999 non-null int64
874965758    79999 non-null int64
dtypes: int64(4)
memory usage: 2.4 MB


,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [8]:
test_setDF.info()
test_setDF.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 4 columns):
1            19999 non-null int64
6            19999 non-null int64
5            19999 non-null int64
887431973    19999 non-null int64
dtypes: int64(4)
memory usage: 625.0 KB


,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [9]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print (nb_users, nb_movies)

943 1682


In [0]:
# Converting the data into an array with users in lines and movies in columns

def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
  

training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
# Creating the architecture of the Neural Network
device = torch.device("cuda:0")
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
sae.cuda() #making sure GPU Cuda is used
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [0]:
#torch.cuda.is_available()
#torch.cuda.device_count()
#cudnn.benchmark = True

In [17]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        input = input.to(device) #important
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 193 loss: 0.9138549039585948
epoch: 194 loss: 0.9132874003528219
epoch: 195 loss: 0.913618023139293
epoch: 196 loss: 0.9129193539771284
epoch: 197 loss: 0.9132501663659316
epoch: 198 loss: 0.912156874115787
epoch: 199 loss: 0.9128639085031011
epoch: 200 loss: 0.91242956599042


In [27]:
test_loss = 0
s = 0

for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    input = input.to(device) #important
    target = Variable(test_set[id_user]).unsqueeze(0)
    target = target.to(device)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print('test_loss: '+str(test_loss/s))

test_loss: 0.9527670665918379
